In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import os
import json
from  matplotlib.ticker import FuncFormatter

warnings.filterwarnings('ignore')

In [2]:
custom_palette = 'Paired'
sns.set_palette(custom_palette)


def set_directories(similarity, dataset):
    print('SET_DIRECTORIES...')
    general_directory = '../plots/reordering/' + dataset + '/' + similarity + '/'
    general_results_directory = '../results/reordering/' + dataset + '/' + similarity + '/'

    # if the directory does not exist, then create it
    if not os.path.exists(general_directory):
        os.makedirs(general_directory)

    return general_directory, general_results_directory


def retrieve_from_pf(dataset, label_ls_initial):
    dir = '../results/pretrained_finetuned/' + dataset

    oracle_df = pd.DataFrame(columns=['model_name', 'reorder_labels', 'descending_order', 'labels_query_order', 'matched', 'accuracy'])

    # for models in dir
    for model in os.listdir(dir):
        model_path = os.path.join(dir, model)
        df = pd.read_csv(model_path)

        descending_oracle = len(df[df['0'] == 1])
        num_last_label = str(len(label_ls_initial) - 1)
        ascending_oracle = len(df[df[num_last_label] == 1])

        oracle_df.loc[len(oracle_df)] = [model, 2, 1, 1, descending_oracle, descending_oracle/len(df)]
        oracle_df.loc[len(oracle_df)] = [model, 2, 0, 1, ascending_oracle, ascending_oracle/len(df)]
    
    return oracle_df
    

def df_model_reordering(general_results_directory, oracle_df, general_directory):
    print('DF_MODEL_REORDERING...')
    
    df = pd.read_csv(general_results_directory + 'results_reordering.csv')

    df['reorder_labels'] = df['reorder_labels'].apply(lambda x: 0 if x == False else 1)

    df = pd.concat([df, oracle_df], ignore_index=True)

    df['model_name'] = df['model_name'].str.replace('meta-llama', '').str.replace('mistralai', '').str.replace('/', '').str.replace('_', ' ').str.replace('.csv', '').str.strip()

    # transform the reorder_labels column into a list of integers where 0 when column reorder_labels is 'False', 1 when column reorder_labels is 'True' and column descending_order is 'False', 2 when column reorder_labels is 'True' and column descending_order is 'True'
    df_tmp = df.copy()
    df_tmp['reorder_labels3'] = ''
    for i, x in df_tmp.iterrows():
        if df_tmp['reorder_labels'][i] == 2:
            if df_tmp['descending_order'][i] == 0:
                df_tmp['reorder_labels3'][i] = 'Ascending Oracle'
            else:
                df_tmp['reorder_labels3'][i] = 'Descending Oracle'
        elif df_tmp['reorder_labels'][i] == 0:
            df_tmp['reorder_labels3'][i] = 'No reordering'
        elif df_tmp['descending_order'][i] == 0:
            df_tmp['reorder_labels3'][i] = 'Ascending reordering'
        elif df_tmp['descending_order'][i] == 1:
            df_tmp['reorder_labels3'][i] = 'Descending reordering'
        elif df_tmp['descending_order'][i] == 2:
            df_tmp['reorder_labels3'][i] = 'Descending-Ascending reordering'
        elif df_tmp['descending_order'][i] == 3:
            df_tmp['reorder_labels3'][i] = 'Model-bias reordering'
    df = df_tmp.copy()
    df = df.sort_values(by=['model_name', 'reorder_labels3'], ascending=True)
    df = df.rename(columns={'reorder_labels3': 'reordering'})
    df = df.sort_values(by=['model_name'], ascending=True)


    df2 = df.copy()
    df2 = df2[df2['labels_query_order'] == 1]
    df2 = df2.drop(columns=['reorder_labels', 'descending_order', 'labels_query_order', 'matched'])

    df2 = df2[(df2['reordering'] != 'Ascending reordering') & (df2['reordering'] != 'Ascending Oracle')]

    # Create pivot table
    pivot_table = df2.pivot_table(
        values='accuracy', 
        index='model_name', 
        columns='reordering'
    )

    display(pivot_table)

    _, ax = plt.subplots(figsize=(4.5, 5))
    ax = sns.heatmap(pivot_table, ax=ax, cmap=sns.color_palette("Blues", as_cmap=True), annot=True, fmt='.2f', vmin=0, vmax=1, annot_kws={'color': 'black'})
    ax.set_xlabel('')
    ax.set_ylabel('')

    plt.tight_layout()
    plt.savefig(general_directory + 'accuracy.pdf', bbox_inches='tight', dpi=300)
    plt.show()


    df = df[df['reorder_labels']<2]
    return df


def plot_model_reordering(df, general_directory):
    print('PLOT_MODEL_REORDERING...')
    # remove from model_name the prefix 'meta-llama/' or prefix 'mistralai/'
    df['model_name'] = df['model_name'].str.replace('meta-llama', '').str.replace('mistralai', '').str.replace('/', '').str.replace('_', ' ').str.strip()

    # order df based on the column reordering in the following order: 'Ascending reordering', 'Descending reordering', 'No reordering', 'Descending-Ascending reordering', 'Model-bias reordering'
    df = df.sort_values(by=['model_name','reordering'], ascending=True)

    # filter the dataframe based on the descending_order column, if reorder_labels is True and descending_order is 0, then remove the row
    df_tmp = df[(df['labels_query_order'] == 0)]
    df1 = df_tmp

    df_tmp = df[(df['labels_query_order'] == 1)]
    df2 = df_tmp
    
    # print for each model the matched column values with respect to prompt_question_order. for each model i want to show a barplot with hue from reorder_labels
    _, ax = plt.subplots(1,2, figsize=(10, 3), sharey=True, sharex=True)

    sns.barplot(data=df2, x='model_name', y='matched', hue='reordering', ax=ax[0])
    ax[0].set_title('Options then Question')
    plt.setp(ax[0].get_xticklabels(), rotation=90)
    ax[0].set_xlabel('')

    sns.barplot(data=df1, x='model_name', y='matched', hue='reordering', ax=ax[1])
    ax[1].set_title('Question then Options')
    plt.setp(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel('')

    # show legend only for the first plot
    ax[0].legend(loc='upper left', title='Reordering', bbox_to_anchor=(0, 1.75))
    ax[1].legend(loc='upper left', title='Reordering', bbox_to_anchor=(0.5, 1.75))
    ax[1].get_legend().remove()

    plt.tight_layout()
    plt.savefig(general_directory + 'reordering_overview.pdf', bbox_inches='tight', dpi=300)
    plt.show()

def plot_model_reordering_onecolumn(df, general_directory):
    print('PLOT_MODEL_REORDERING...')
    # remove from model_name the prefix 'meta-llama/' or prefix 'mistralai/'
    df['model_name'] = df['model_name'].str.replace('meta-llama', '').str.replace('mistralai', '').str.replace('/', '').str.replace('_', ' ').str.strip()

    df_tmp = df[(df['labels_query_order'] == 1)]
    df2 = df_tmp
    
    # print for each model the matched column values with respect to prompt_question_order. for each model i want to show a barplot with hue from reorder_labels
    _, ax = plt.subplots(1,1, figsize=(5, 3), sharey=True, sharex=True)

    sns.barplot(data=df2, x='model_name', y='matched', hue='reordering', ax=ax)
    #ax.set_title('')
    plt.setp(ax.get_xticklabels(), rotation=90)
    ax.set_xlabel('')

    # show legend only for the first plot
    ax.legend(loc='upper left', title='Reordering', bbox_to_anchor=(0, 1.65))

    plt.tight_layout()
    plt.savefig(general_directory + 'reordering_overview_onecolumn.pdf', bbox_inches='tight', dpi=300)
    plt.show()


def plot_similarity_reordering(model, general_results_directory, similarity, general_directory):
    print('PLOT_SIMILARITY_REORDERING...')
    fontsize = 12
    file = general_results_directory + model + '.csv'
    if not os.path.exists(file):
        return None
    df = pd.read_csv(general_results_directory + model + '.csv')


    descending_order = 1
    df_distribution = df[df['descending_order'] == descending_order]
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5), sharey=False)

    # barplot for each x as label position plot y as count
    sns.countplot(x='target_label_id', data=df_distribution, ax=ax, legend=False, color=sns.color_palette("Paired")[1])
    ax.set_xlabel('Label position', fontsize=fontsize)
    ax.set_ylabel('Count', fontsize=fontsize)
    ax.set_title(similarity.replace('_', ' ').capitalize(), fontsize=fontsize+2)

    # set x labels from 0 to len of labels with step 10
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
    x_len = len(df['target_label'].unique())
    ax.set_xticks(range(0, x_len, 10))
    ax.set_xticklabels(range(0, x_len, 10), fontsize=fontsize)

    plt.tight_layout()
    plt.savefig(general_directory + model + '_distribution_target_label_' + similarity + '.pdf', bbox_inches='tight', dpi=300)
    plt.show()


    
    # display(df_distribution.head())
    k_values=[1, 5, 10, 20, 30, 50, int(max(df_distribution['target_label_id']))]
    total_samples = len(df_distribution)
    print("Total samples:", total_samples)

    top_k_df = pd.DataFrame(columns=['Top-K', 'Percentage', 'Count'])
    for k in k_values:
        count_in_top_k = (df_distribution['target_label_id'] < k).sum()
        percentage = (count_in_top_k / total_samples) * 100
        top_k_df.loc[len(top_k_df)] = [f"Top-{k}", f"{percentage:.2f}%", f"{count_in_top_k}/{total_samples}"]

    # Convert to DataFrame for display
    display(top_k_df)

    return df


def concatenate_pf_reordering(df):
    print('CONCATENATE_PF_REORDERING...')
    if df is None:
        return None

    # remove rows where labels_query_order is 0
    df = df[df['labels_query_order'] == 1]

    # Create a column reordering with "No reordering" if reorder_labels is 0, "Ascending reordering" if reorder_labels is 1 and descending_order is 0, "Descending reordering" if reorder_labels is 1 and descending_order is 1
    df['reordering'] = ['No reordering' if x == 0 else 'Ascending reordering' if x == 1 and y == 0 else 'Descending reordering' for x, y in zip(df['reorder_labels'], df['descending_order'])]

    # print number of samples where matched is 1 in df for No reordering
    print('Number of samples where matched is 1 for No reordering:', len(df[(df['reordering'] == 'No reordering') & (df['matched'] == 1)]))
    # print number of samples where matched is 1 in df for Ascending reordering
    print('Number of samples where matched is 1 for Ascending reordering:', len(df[(df['reordering'] == 'Ascending reordering') & (df['matched'] == 1)]))
    # print number of samples where matched is 1 in df for Descending reordering
    print('Number of samples where matched is 1 for Descending reordering:', len(df[(df['reordering'] == 'Descending reordering') & (df['matched'] == 1)]))
    # print number of rows in df for No reordering
    print('Number of rows in df for No reordering:', len(df[df['reordering'] == 'No reordering']))
    # print number of rows in df for Ascending reordering
    print('Number of rows in df for Ascending reordering:', len(df[df['reordering'] == 'Ascending reordering']))
    # print number of rows in df for Descending reordering
    print('Number of rows in df for Descending reordering:', len(df[df['reordering'] == 'Descending reordering']))

    return df


def plot_reordering_quadrants(df, general_directory, model, label_ls_initial):
    print('PLOT_REORDERING_QUADRANTS...')
    if df is None:
        return None

    # remove rows where matched is 0
    df = df[df['matched'] == 1]

    df = df.sort_values(by=['reordering'], ascending=True)

    # total number of possible positions for the target_label
    total_positions = len(label_ls_initial)
    # split total_positions into 4 variables
    first_quadrant_positions = total_positions // 4
    second_quadrant_positions = total_positions // 2
    third_quadrant_positions = total_positions // 4 * 3

    first_quadrant = label_ls_initial[:first_quadrant_positions]
    second_quadrant = label_ls_initial[first_quadrant_positions:second_quadrant_positions]
    third_quadrant = label_ls_initial[second_quadrant_positions:third_quadrant_positions]
    fourth_quadrant = label_ls_initial[third_quadrant_positions:]
    _, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15), sharey=True, sharex=False)
    sns.countplot(x='target_label', data=df[df['target_label'].isin(first_quadrant)], hue='reordering', ax=ax[0][0])
    ax[0][0].set_xlabel('')
    ax[0][0].set_ylabel('')
    sns.countplot(x='target_label', data=df[df['target_label'].isin(second_quadrant)], hue='reordering', ax=ax[0][1], legend=False)
    ax[0][1].set_xlabel('')
    ax[0][1].set_ylabel('')
    sns.countplot(x='target_label', data=df[df['target_label'].isin(third_quadrant)], hue='reordering', ax=ax[1][0], legend=False)
    ax[1][0].set_xlabel('')
    ax[1][0].set_ylabel('')
    sns.countplot(x='target_label', data=df[df['target_label'].isin(fourth_quadrant)], hue='reordering', ax=ax[1][1], legend=False)
    ax[1][1].set_xlabel('')
    ax[1][1].set_ylabel('')

    ax[0][0].tick_params(axis='x', rotation=90)
    ax[0][1].tick_params(axis='x', rotation=90)
    ax[1][0].tick_params(axis='x', rotation=90)
    ax[1][1].tick_params(axis='x', rotation=90)

    ax[0][0].legend(loc='upper left', title='Reordering', bbox_to_anchor=(0, 1.2))

    plt.tight_layout()
    plt.xticks(rotation=90)
    plt.savefig(general_directory + model + '_reordering_quadrants.pdf', bbox_inches='tight', dpi=300)
    plt.show()



In [3]:
def call_functions(similarity, model, label_ls_initial, dataset):
    general_directory, general_results_directory = set_directories(similarity, dataset)
    oracle_df = retrieve_from_pf(dataset, label_ls_initial)
    df = df_model_reordering(general_results_directory, oracle_df, general_directory)
    plot_model_reordering(df, general_directory)
    plot_model_reordering_onecolumn(df, general_directory)
    df = plot_similarity_reordering(model, general_results_directory, similarity, general_directory)
    df = concatenate_pf_reordering(df)
    plot_reordering_quadrants(df, general_directory, model, label_ls_initial)

In [ ]:
""" similarities = ['cosine_similarity', 'euclidean_distance', 'manhattan_distance']
models = ['meta-llama_Llama-2-7b-chat-hf', 'meta-llama_Llama-2-13b-chat-hf', 'meta-llama_Meta-Llama-3-8B-Instruct', 'mistralai_Mistral-7B-Instruct-v0.3']
datasets = ['CLINC150_subset', 'BANKING77', 'HWU64'] """

similarities = ['cosine_similarity']
models = ['meta-llama_Meta-Llama-3-8B-Instruct', 'mistralai_Mistral-7B-Instruct-v0.3']
datasets = ['CLINC150_subset', 'BANKING77', 'HWU64']

for similarity in similarities:
    for model in models:
        for dataset in datasets:
            label_ls_initial = None
            # read labels from json file
            with open('../Dataset/' + dataset + '_labels.json', 'r') as f:
                label_ls_initial = json.load(f)

            call_functions(similarity, model, label_ls_initial, dataset)

In [16]:
def plot_similarities(model, dataset, similarity1, similarity2, similarity3):
    fontsize = 16
    file1 = '../results/reordering/' + dataset + '/' + similarity1 + '/' + model + '.csv'
    file2 = '../results/reordering/' + dataset + '/' + similarity2 + '/' + model + '.csv'
    file3 = '../results/reordering/' + dataset + '/' + similarity3 + '/' + model + '.csv'

    if not os.path.exists(file1) or not os.path.exists(file2) or not os.path.exists(file3):
        return None

    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    df3 = pd.read_csv(file3)

    descending_order = 1

    df_distribution1 = df1[df1['descending_order'] == descending_order]
    df_distribution2 = df2[df2['descending_order'] == descending_order]
    df_distribution3 = df3[df3['descending_order'] == descending_order]

    _, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 3), sharey=True)

    sns.countplot(x='target_label_id', data=df_distribution1, ax=ax[0], legend=False, color=sns.color_palette("Paired")[1])
    sns.countplot(x='target_label_id', data=df_distribution2, ax=ax[1], legend=False, color=sns.color_palette("Paired")[1])
    sns.countplot(x='target_label_id', data=df_distribution3, ax=ax[2], legend=False, color=sns.color_palette("Paired")[1])

    ax[0].set_xlabel('Label position', fontsize=fontsize)
    ax[1].set_xlabel('Label position', fontsize=fontsize)
    ax[2].set_xlabel('Label position', fontsize=fontsize)

    ax[0].set_ylabel('Count', fontsize=fontsize)
    ax[1].set_ylabel('')
    ax[2].set_ylabel('')

    ax[0].set_title(similarity1.replace('_', ' ').capitalize(), fontsize=fontsize+2)
    ax[1].set_title(similarity2.replace('_', ' ').capitalize(), fontsize=fontsize+2)
    ax[2].set_title(similarity3.replace('_', ' ').capitalize(), fontsize=fontsize+2)

    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))  
    x_len = len(df1['target_label'].unique())

    ax[0].set_xticks(range(0, x_len, 50))
    ax[1].set_xticks(range(0, x_len, 50))
    ax[2].set_xticks(range(0, x_len, 50))

    ax[0].set_xticklabels(range(0, x_len, 50), fontsize=fontsize)
    ax[1].set_xticklabels(range(0, x_len, 50), fontsize=fontsize)
    ax[2].set_xticklabels(range(0, x_len, 50), fontsize=fontsize)

    plt.tight_layout()
    plt.savefig('../plots/reordering/' + dataset + '/similarities.pdf', bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
similarities = ['cosine_similarity', 'euclidean_distance', 'manhattan_distance']
model = 'meta-llama_Meta-Llama-3-8B-Instruct'
dataset = 'CLINC150_subset'

plot_model_reordering_onecolumn(df, general_directory)
plot_similarities(model, dataset, similarities[0], similarities[1], similarities[2])